In [8]:
import sys
import logging

In [15]:
logger = logging.getLogger("myJupyter")
logger.setLevel(logging.DEBUG)

ch = logging.StreamHandler(sys.stdout)
ch.setLevel(logging.ERROR)

formatter = logging.Formatter(
    "%(levelname)s: [%(asctime)s]($(name)s) %(message)s"
)
ch.setFormatter(formatter)

logger.addHandler(ch)

In [16]:
logger.info("Info log")

INFO:myJupyter:Info log


In [17]:
try:
    raise ValueError()
except:
    print("Error")
else:
    print("else")
finally:
    print("finally")

Error
finally


In [18]:
def func():
    try:
        raise ValueError()
    except:
        print("Error")
        raise
    else:
        print("Else")
        return 1
    finally:
        print("finally")

In [19]:
func()

Error
finally


ValueError: 

In [25]:
import inspect
import logging
import types

from functools import wraps


class MetaLogger:
    @classmethod
    def add_log_decorator(cls, attribute, attribute_name):
        '''
        Decoration function which is used for logging every function
        '''
        @wraps(attribute)
        def wrapper(*args, **kwargs):
            logger = cls.logger
            state = "Success"
            try:
                result = attribute(*args, **kwargs)
            except:
                state = "Failed"
                raise
            else:
                return result
            finally:
                logger.log("%s (%s)", attribute_name, state)
        return wrapper

    @staticmethod
    def update_class_methods(cls):
        if not hasattr(cls, "__decorated"):
            for attr_name, attr in inspect.getmembers(cls):
                if isinstance(attr, types.FunctionType):
                    setattr(
                        cls,
                        attr_name,
                        MetaLogger.add_log_decorator(
                            attr,
                            cls.__name__ + '.' + attr_name
                        )
                    )
        cls.__decorated = True

    def __new__(cls, *args, **kwargs):
        if not hasattr(cls, "logger"):
            cls.logger = logging.getLogger(__name__)
            cls.logger.setLevel(logging.DEBUG)
        MetaLogger.update_class_methods(cls)
        instance = super(MetaLogger, cls).__new__(cls)
        return instance

In [26]:
class Foo(MetaLogger):
    def __init__(self):
        print("Creating foo")
    
    def bar(self):
        print("foo.bar")

In [27]:
foo = Foo()

AttributeError: type object 'MetaLogger' has no attribute 'logger'

In [1]:
import inspect
import logging
import types

from functools import wraps


class MetaLogger(type):
    def __new__(cls, name, base, dct):
        _type = super().__new__(cls, name, base, dct)
        _type.logger = logging.getLogger(name)
        _type.logger.setLevel(logging.DEBUG)
        MetaLogger.update_class_methods(_type)
        return _type
    
    @staticmethod
    def add_log_decorator(cls, attribute, attribute_name):
        '''
        Decoration function which is used for logging every function
        '''
        @wraps(attribute)
        def wrapper(*args, **kwargs):
            logger = cls.logger
            state = "Success"
            try:
                result = attribute(*args, **kwargs)
            except:
                state = "Failed"
                raise
            else:
                return result
            finally:
                logger.debug("%s (%s)", attribute_name, state)
        return wrapper

    @staticmethod
    def update_class_methods(cls):
        if not hasattr(cls, "__decorated"):
            for attr_name, attr in inspect.getmembers(cls):
                if isinstance(attr, types.FunctionType):
                    setattr(
                        cls,
                        attr_name,
                        MetaLogger.add_log_decorator(
                            cls,
                            attr,
                            cls.__name__ + '.' + attr_name
                        )
                    )
        cls.__decorated = True

In [2]:
try:
    handler = handler
except NameError:
    import sys
    handler = logging.StreamHandler(sys.stdout)
    formatter = logging.Formatter(
        "%(levelname)s: [%(asctime)s](%(name)s) %(message)s"
    )
    handler.setFormatter(formatter)
    logger = logging.getLogger()
    logger.addHandler(handler)
finally:
    handler = handler

In [6]:
class Foo(metaclass=MetaLogger):
    def __init__(self):
        print("Create Foo")
    
    def bar(self):
#         raise ValueError("foo.bar fails")
        print("Execute foo.bar")
        

foo = Foo()
foo.bar()

Create Foo
DEBUG: [2021-07-19 15:32:19,278](Foo) Foo.__init__ (Success)
Execute foo.bar
DEBUG: [2021-07-19 15:32:19,280](Foo) Foo.bar (Success)
